This is a companion notebook for [arXiv:1512.06860](https://arxiv.org/abs/1512.06860). 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyquil import Program
from pyquil.api import WavefunctionSimulator
from pyquil.paulis import sI, sX, sY, sZ, PauliTerm

wfs = WavefunctionSimulator()

In [ ]:
pauli_terms = [sI(0)*sI(1), sZ(0)*sI(1), sI(0)*sZ(1), sZ(0)*sZ(1), sY(0)*sY(1), sX(0)*sX(1)]

for term in pauli_terms:
    print(term.pauli_string())

In [ ]:
def build_program_wo_ro_measure():
    program = Program("""
# declare placeholder for continuously parameterized RZ rotation angle
DECLARE theta REAL

# set up initial state
X 0

# build the exponentiated operator
RX(-pi/2) 0
RY(pi/2) 1

CNOT 1 0
RZ(theta) 0
CNOT 1 0

RX(pi/2) 0
RY(-pi/2) 1""")
    return program

program_wo_ro_measure = build_program_wo_ro_measure()
print(program_wo_ro_measure)

## recreating Figure 2a

In [ ]:
thetas = np.linspace(-np.pi, np.pi, 25)

pauli_term_evs = [[]]*len(pauli_terms)
for theta in thetas:
    pauli_term_expectation_values = wfs.expectation(program_wo_ro_measure,
                                                    pauli_terms,
                                                    memory_map={'theta': [theta]})
    pauli_term_evs = list(map(lambda x,y: x + [y], pauli_term_evs, pauli_term_expectation_values))

for i in range(len(pauli_terms)):
    plt.plot(thetas, pauli_term_evs[i], label=pauli_terms[i].pauli_string())
    plt.xlabel(r'rotation angle \theta')  # LaTeX why
    plt.ylabel('expectation value')
    plt.legend()

## recreating Figure 2b

In [ ]:
thetas = np.linspace(-np.pi, np.pi, 100)
bond_lengths = np.linspace(0.20, 2.85, 54) # values determined by Hamiltonian coefficients in Table 1

hamiltonain_coefficients = pd.read_csv('vqe_h2.csv', index_col=0, header=None)

hamiltonian_evs = np.zeros((len(thetas), len(bond_lengths)))
for i, theta in enumerate(thetas):
    pauli_term_expectation_values = wfs.expectation(program_wo_ro_measure,
                                                    pauli_terms,
                                                    memory_map={'theta': [theta]})
    for j, R in enumerate(bond_lengths):
        hamiltonian_weights = list(hamiltonain_coefficients.loc[round(R, 2)])
        # dot the Pauli term expectation values vector with the Hamiltonian weights vector, which is a 
        # function of bond length R
        hamiltonian_expectation_value = np.dot(pauli_term_expectation_values, hamiltonian_weights)
        hamiltonian_evs[i][j] = hamiltonian_expectation_value
        
plt.figure(figsize=(10,10))
plt.imshow(hamiltonian_evs)
plt.colorbar()
plt.xlabel('R: bond_length (angstroms)')
plt.ylabel(r'\theta (radians)')
plt.show()

## recreating Figure 3a

In [ ]:
min_energies = []
for col in range(hamiltonian_evs.shape[1]):
    energies = list(hamiltonian_evs[:,col])
    min_energy = min(energies)
    min_energies.append(min_energy)
    theta_min_ev = energies.index(min(energies))
    
plt.plot(bond_lengths, min_energies)
plt.show()